In [7]:
import io
import subprocess
import os
import glob

import pandas as pd
import numpy as np
import scipy
import scipy.stats as stats
from sklearn.decomposition import PCA
import pickle
import re
import math
import plotnine as pn
import seaborn as sns
import glob
import subprocess
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from numpy import log
from collections import defaultdict,Counter
import re
import statsmodels
import warnings

# !pip install statannotations==0.4.0
# #!pip uninstall statannotations --yes
# from statannotations.Annotator import Annotator

warnings.filterwarnings("ignore")
pd.options.display.max_rows = None
pd.options.display.max_columns=None
pd.options.display.max_colwidth=None
work_dir="/home/hazhang/projects/VHL_promoter_methylation_12122023"

# Data processing

In [14]:
#to get mcm cancer-free sample (code from Kyle)
# meta file of all methyl samples
meta_raw = pd.read_csv("/ghess/omni_v2/users/schen/methylation_data/all-methyl-eap-dev-training-meta.20230828.all_samples.tsv", sep="\t")
meta_raw = meta_raw.drop_duplicates(subset=["sample_id"])
norm = meta_raw.query('comment.isna() and SUCCESS != "FAIL" and cancer_type == "CANCER_FREE" and subset=="mcm" and cram_path.notna()')
norm.head()

,sample_id,runid,ctrl_cor_frac,pos_ctrl_after_correction,pred_type,CRC,LUNG,BREAST,BLADDER,AGGREGATED,CRC_TF,LUNG_TF,BREAST_TF,BLADDER_TF,AGGREGATED_TF,cohort,request_id,patient_id,visit_name,Alt-NoAlt,SUCCESS,Sample_comment,max_maf_pct_report,RUO_methylation_status,RUO_methylation_tf,TMB_score,TMB_category,MSI_High,input_ng,Plasma_ml_input,Bloodcoll_date,raw_cancer_type,cancer_type,cfDNA_ng_yield,subset,genomic_method,fc_name,cram_path,somatic_max_maf_gene,mut_aa,somatic_max_maf_pct,is_titration,sample_pool,batch,category,gender,age,subtype,treatment,recur,baseline_cancer_type,comment,qc,stage,ethnicity,screening_method,ER_status,HER2_status,ER/HER2_method,comorbidity,smoking_status,s3_panelwide_gene,s3_aa,somatic_maxmafpct_panelwide,somatic_source,mcm_gene,mcm_aa,g360_gene,g360_aa,somatic_driver_pct,somatic_driver_gene,hyper_residual_count_pos_ctrl,panel,msi_status,msi_score
14125,B079640-2,NaN,1.0,52402.0,cancer_free,-12.80378,-9.88626,-12.31540,-5.55190,-5.55190,0.0,0.0,0.0,0.0,0.0,CUB_01,NaN,100141685,pre-op,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.150505,NaN,NaN,NaN,CANCER_FREE,NaN,mcm,Shield,NaN,/ghsfa/screening/flowcells/processed/BIP4/devel/220511_A01021_0460_BHHJJCDSX3.devel_methylome_v2_8.MethylomeT123v2_set6barcodes.220513_153914/B079640-2.pre-realigned.cram,NaN,NaN,NaN,no,NaN,LTO_0521,other,Female,49.0,NaN,NaN,NaN,NaN,NaN,SUCCESS,grade_6,NaN,colonscopy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MCM,NaN,NaN
14147,FB09162087-2,NaN,1.0,185722.0,cancer_free,-7.04119,-5.70565,-9.95082,-3.08714,-7.04119,0.0,0.0,0.0,0.0,0.0,ALM_CRC,NaN,106-3061,pre-op,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.980519,NaN,NaN,NaN,CANCER_FREE,NaN,mcm,Shield,NaN,/ghsfa/screening/flowcells/processed/BIP4/devel/220511_A01021_0460_BHHJJCDSX3.devel_methylome_v2_8.MethylomeT123v2_set6barcodes.220513_153914/FB09162087-2.pre-realigned.cram,NaN,NaN,NaN,no,NaN,LTO_0521,other,NaN,45.0,NaN,NaN,NaN,NaN,NaN,SUCCESS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MCM,NaN,NaN
14148,FB09162093-2,NaN,1.0,64462.0,cancer_free,-14.99297,-6.19359,-8.86810,-5.36549,-6.19359,0.0,0.0,0.0,0.0,0.0,ALM_CRC,NaN,116-3026,pre-op,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.458196,NaN,NaN,NaN,CANCER_FREE,NaN,mcm,Shield,NaN,/ghsfa/screening/flowcells/processed/BIP4/devel/220511_A01021_0460_BHHJJCDSX3.devel_methylome_v2_8.MethylomeT123v2_set6barcodes.220513_153914/FB09162093-2.pre-realigned.cram,NaN,NaN,NaN,no,NaN,LTO_0521,other,NaN,51.0,NaN,NaN,NaN,NaN,NaN,SUCCESS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MCM,NaN,NaN
14158,FB09630376-2,NaN,1.0,76634.0,cancer_free,-11.49278,-10.28273,-14.57369,-8.91455,-11.49278,0.0,0.0,0.0,0.0,0.0,ALM_CRC,NaN,104-3036,pre-op,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.607516,NaN,NaN,NaN,CANCER_FREE,NaN,mcm,Shield,NaN,/ghsfa/screening/flowcells/processed/BIP4/devel/220511_A01021_0460_BHHJJCDSX3.devel_methylome_v2_8.MethylomeT123v2_set6barcodes.220513_153914/FB09630376-2.pre-realigned.cram,NaN,NaN,NaN,no,NaN,LTO_0521,other,NaN,56.0,NaN,NaN,NaN,NaN,NaN,SUCCESS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MCM,NaN,NaN
14165,FB09630587-2,NaN,1.0,166985.0,cancer_free,-3.94195,-3.08528,-8.08071,-2.39462,-3.94195,0.0,0.0,0.0,0.0,0.0,ALM_CRC,NaN,132-3012,pre-op,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.695346,NaN,NaN,NaN,CANCER_FREE,NaN,mcm,Shield,NaN,/ghsfa/screening/flowcells/processed/BIP4/devel/220511_A01021_0460_BHHJJCDSX3.devel_methylome_v2_8.MethylomeT123v2_set6barcodes.220513_153914/FB09630587-2.pre-realigned.cram,NaN,NaN,NaN,no,NaN,LTO_0521,other,NaN,78.0,NaN,NaN,NaN,NaN,NaN,SUCCESS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MCM,NaN,NaN


In [3]:
norm.shape

(4286, 75)

In [15]:
#edit the 'cram_path' column to generate "mbd_hyper_file_path" column for each sample's mbd_hyper_molecules.tsv
df_norm = norm.copy()

df_norm['mbd_hyper_file_path'] = df_norm['cram_path'].str.replace(r'\.pre-realigned\.cram$', '.mbd_hyper_molecules.tsv', regex=True)
# df_norm.to_csv('df_norm.csv', index=False)
# df_norm.head()

# Assuming df_norm is your DataFrame

# Calculate the number of rows to split at
split_row = len(df_norm) // 2

# Split the DataFrame into two halves
df_norm_1 = df_norm.iloc[:split_row]
df_norm_2 = df_norm.iloc[split_row:]

# Save each half to a separate CSV file
df_norm_1.to_csv('df_norm_1.csv', index=False)
df_norm_2.to_csv('df_norm_2.csv', index=False)

In [ ]:
#The function to calculate MB score
mb_model_coef = pd.read_csv("/home/yhe/repo/bip-products/models/Lunar2-0.2/methylation_caller/mafband_v2023_06_30/msre_mr_model.tsv", sep="\t")

def calculate_MB_score(msre_file, mb_model_coef):
    msre_df = pd.read_table(msre_file)
    msre_df["region_score"] = np.log10(msre_df["hyper_residual_count"]/msre_df["hyper_residual_count_pos_ctrl"] + 1e-5)
    v6_score_df = (
        msre_df
        .merge(mb_model_coef)
        .groupby(["run_sample_id", "runid"])
        .apply(lambda x: (x["region_score"] * x["weight"]).sum() + x["bias"].iloc[0])
        .reset_index(name="v2_mb_score")
    )
    v2_score = v6_score_df.v2_mb_score.values[0]

    return v2_score
    
v2_score = calculate_MB_score("/ghess/omni_v2/users/schen/mafband_on_s3/mafband_shield_bip/221208_A01810_0059_AHKW77DSX5_PROD148_AP1.Sirius-1.0.0-RLS/D8_S3P_0988/D8_S3P_0988.msre_caller_mr_features.hdr.tsv.gz", mb_model_coef)

print(v2_score)

In [13]:
df_norm = df_norm.head(2500)

In [ ]:
# def calculate_methylation_detected(file_path):
#     # Check if the file exists
#     if not os.path.exists(file_path):
#         print(f"No file found at {file_path}")
#         return np.nan

#     try:
#         # Read the file into a DataFrame
#         df = pd.read_table(file_path)

#         # Apply the filtering conditions
#         filtered_df = df[(df['chrom'] == 3) &
#                          (df['start'] >= 10178462) &
#                          (df['start'] <= 10183462) &
#                          ((df['end'] - df['start']) < 400) &
#                          (df['mean_mapq'] >= 60) &
#                          (df['cg_count'] >= 10)]
        
#         # Calculate the sum of 'num_molecules'
#         sum_num_molecules = filtered_df['num_molecules'].sum()

#         # Return 1 if sum > 10, else 0
#         return 1 if sum_num_molecules > 10 else 0

#     except Exception as e:
#         print(f"Error processing file {file_path}: {e}")
#         return np.nan

# # Apply the function to each row and assign the result to a new column
# df_norm['VHL_promoter_methylation_detected'] = df_norm['mbd_hyper_file_path'].apply(calculate_methylation_detected)
# df_norm.head()

In [ ]:
def calculate_methylation_detected_and_methyl_score(mbd_hyper_file_path, sample_id):
    # Check if the mbd_hyper file exists
    if not os.path.exists(mbd_hyper_file_path):
        print(f"No file found at {mbd_hyper_file_path}")
        return np.nan, np.nan, np.nan

    try:
        # Read the mbd_hyper file into a DataFrame
        df_mbd_hyper = pd.read_table(mbd_hyper_file_path)

        # Apply the primary filtering conditions
        filtered_df = df_mbd_hyper[(df_mbd_hyper['chrom'] == 3) &
                                   (df_mbd_hyper['start'] >= 10178462) &
                                   (df_mbd_hyper['start'] <= 10183462) &
                                   ((df_mbd_hyper['end'] - df_mbd_hyper['start']) < 400) &
                                   (df_mbd_hyper['mean_mapq'] >= 60) &
                                   (df_mbd_hyper['cg_count'] >= 10)]
        
        # Calculate the sum of 'num_molecules'
        sum_num_molecules = filtered_df['num_molecules'].sum()

        # Define the path for the secondary file
        msre_call_rerun_by_sai = "/ghess/omni_v2/users/schen/methylation_data/msre_call/"
        secondary_file_name = f"{sample_id}.msre_all_call.hdr.tsv"

        subfolder = os.popen(f"ls -d {msre_call_rerun_by_sai}*/{sample_id}").read().strip()
        secondary_file_path = os.path.join(subfolder, secondary_file_name)

        # Check if the secondary file exists
        if not os.path.exists(secondary_file_path):
            print(f"No secondary file found at {secondary_file_path}")
            return 0, sum_num_molecules, np.nan

        # Read the secondary file
        df_secondary = pd.read_table(secondary_file_path)
        hyper_residual_count_pos_ctrl = df_secondary['hyper_residual_count_pos_ctrl'].iloc[0]

        # Calculate methyl_score (just for VHL region)
        methyl_score = sum_num_molecules / hyper_residual_count_pos_ctrl if hyper_residual_count_pos_ctrl else np.nan

        # Determine the value for 'VHL_promoter_methylation_detected'
        methylation_detected = 1 if methyl_score > 8.3e-05 and sum_num_molecules > 10 else 0
        return methylation_detected, sum_num_molecules, methyl_score

    except Exception as e:
        print(f"Error processing file {mbd_hyper_file_path}: {e}")
        return np.nan, np.nan, np.nan

results = df_norm.apply(lambda row: calculate_methylation_detected_and_methyl_score(row['mbd_hyper_file_path'], row['sample_id']), axis=1)
df_norm['VHL_promoter_methylation_detected'], df_norm['sum_VHL_promoter_mol'], df_norm['methyl_score_VHL'] = zip(*results)

#save the file as csv
df_norm.to_csv('df_norm_vhl_all.csv', index=False)
df_norm.head()

In [16]:
import os
os.system("sbatch VHL_promoter_methyl.job")

Submitted batch job 70783655


0

In [10]:
# # Writing the function and execution code to a Python script
# python_script = """
# def calculate_methylation_detected_and_methyl_score(mbd_hyper_file_path, sample_id):
#     # Check if the mbd_hyper file exists
#     if not os.path.exists(mbd_hyper_file_path):
#         print(f"No file found at {mbd_hyper_file_path}")
#         return np.nan, np.nan, np.nan

#     try:
#         # Read the mbd_hyper file into a DataFrame
#         df_mbd_hyper = pd.read_table(mbd_hyper_file_path)

#         # Apply the primary filtering conditions
#         filtered_df = df_mbd_hyper[(df_mbd_hyper['chrom'] == 3) &
#                                    (df_mbd_hyper['start'] >= 10178462) &
#                                    (df_mbd_hyper['start'] <= 10183462) &
#                                    ((df_mbd_hyper['end'] - df_mbd_hyper['start']) < 400) &
#                                    (df_mbd_hyper['mean_mapq'] >= 60) &
#                                    (df_mbd_hyper['cg_count'] >= 10)]
        
#         # Calculate the sum of 'num_molecules'
#         sum_num_molecules = filtered_df['num_molecules'].sum()

#         # Define the path for the secondary file
#         msre_call_rerun_by_sai = "/ghess/omni_v2/users/schen/methylation_data/msre_call/"
#         secondary_file_name = f"{sample_id}.msre_all_call.hdr.tsv"

#         subfolder = os.popen(f"ls -d {msre_call_rerun_by_sai}*/{sample_id}").read().strip()
#         secondary_file_path = os.path.join(subfolder, secondary_file_name)

#         # Check if the secondary file exists
#         if not os.path.exists(secondary_file_path):
#             print(f"No secondary file found at {secondary_file_path}")
#             return 0, sum_num_molecules, np.nan

#         # Read the secondary file
#         df_secondary = pd.read_table(secondary_file_path)
#         hyper_residual_count_pos_ctrl = df_secondary['hyper_residual_count_pos_ctrl'].iloc[0]

#         # Calculate methyl_score
#         methyl_score = sum_num_molecules / hyper_residual_count_pos_ctrl if hyper_residual_count_pos_ctrl else np.nan

#         # Determine the value for 'VHL_promoter_methylation_detected'
#         methylation_detected = 1 if methyl_score > 8.3e-05 and sum_num_molecules > 10 else 0
#         return methylation_detected, sum_num_molecules, methyl_score

#     except Exception as e:
#         print(f"Error processing file {mbd_hyper_file_path}: {e}")
#         return np.nan, np.nan, np.nan

# # Assuming df_norm is already defined or loaded here
# df_norm = pd.read_csv('path_to_your_df_norm.csv')

# results = df_norm.apply(lambda row: calculate_methylation_detected_and_methyl_score(row['mbd_hyper_file_path'], row['sample_id']), axis=1)
# df_norm['VHL_promoter_methylation_detected'], df_norm['sum_VHL_promoter_mol'], df_norm['methyl_score_VHL'] = zip(*results)

# df_norm.to_csv('df_norm_vhl_all.csv', index=False)
# """

# with open("calculate_methylation_script.py", "w") as file:
#     file.write(python_script)


In [ ]:
# # Writing the Slurm job script
# slurm_script = """
# #!/bin/bash
# #SBATCH --job-name=calc_methyl
# #SBATCH --output=calc_methyl.log
# #SBATCH --nodes=1
# #SBATCH --ntasks=1
# #SBATCH --time=01:00:00
# #SBATCH --mem=10GB

# module load python3  # Adjust as needed
# python calculate_methylation_script.py
# """

# with open("run_calculation.job", "w") as file:
#     file.write(slurm_script)


In [5]:
# def calculate_methylation_detected_and_methyl_score(mbd_hyper_file_path, sample_id):
#     # Check if the mbd_hyper file exists
#     if not os.path.exists(mbd_hyper_file_path):
#         print(f"No file found at {mbd_hyper_file_path}")
#         return np.nan, np.nan, np.nan

#     try:
#         # Read the mbd_hyper file into a DataFrame
#         df_mbd_hyper = pd.read_table(mbd_hyper_file_path)

#         # Apply the primary filtering conditions
#         filtered_df = df_mbd_hyper[(df_mbd_hyper['chrom'] == 3) &
#                                    (df_mbd_hyper['start'] >= 10178462) &
#                                    (df_mbd_hyper['start'] <= 10183462) &
#                                    ((df_mbd_hyper['end'] - df_mbd_hyper['start']) < 400) &
#                                    (df_mbd_hyper['mean_mapq'] >= 60) &
#                                    (df_mbd_hyper['cg_count'] >= 10)]
        
#         # Calculate the sum of 'num_molecules'
#         sum_num_molecules = filtered_df['num_molecules'].sum()

#         # Define the path for the secondary file
#         msre_call_rerun_by_sai = "/ghess/omni_v2/users/schen/methylation_data/msre_call/"
#         secondary_file_name = f"{sample_id}.msre_all_call.hdr.tsv"

#         subfolder = os.popen(f"ls -d {msre_call_rerun_by_sai}*/{sample_id}").read().strip()
#         secondary_file_path = os.path.join(subfolder, secondary_file_name)

#         # Check if the secondary file exists
#         if not os.path.exists(secondary_file_path):
#             print(f"No secondary file found at {secondary_file_path}")
#             return 0, sum_num_molecules, np.nan

#         # Read the secondary file
#         df_secondary = pd.read_table(secondary_file_path)
#         hyper_residual_count_pos_ctrl = df_secondary['hyper_residual_count_pos_ctrl'].iloc[0]

#         # Calculate methyl_score
#         methyl_score = sum_num_molecules / hyper_residual_count_pos_ctrl if hyper_residual_count_pos_ctrl else np.nan

#         # Determine the value for 'VHL_promoter_methylation_detected'
#         methylation_detected = 1 if methyl_score > 8.3e-05 and sum_num_molecules > 10 else 0
#         return methylation_detected, sum_num_molecules, methyl_score

#     except Exception as e:
#         print(f"Error processing file {mbd_hyper_file_path}: {e}")
#         return np.nan, np.nan, np.nan

# # Function to process a chunk of the DataFrame
# def process_df_chunk(df_chunk):
#     results = df_chunk.apply(lambda row: calculate_methylation_detected_and_methyl_score(row['mbd_hyper_file_path'], row['sample_id']), axis=1)
#     df_chunk['VHL_promoter_methylation_detected'], df_chunk['sum_VHL_promoter_mol'], df_chunk['methyl_score_VHL'] = zip(*results)
#     return df_chunk

# # Function to process the DataFrame in batches
# def process_in_batches(df, batch_size):
#     result_frames = []
#     for start in range(0, len(df), batch_size):
#         end = min(start + batch_size, len(df))
#         df_batch = df[start:end]
#         processed_batch = process_df_chunk(df_batch)
#         result_frames.append(processed_batch)
#     return pd.concat(result_frames)

# # Define batch size
# batch_size = 500 

# # Process df_norm in batches
# df_norm_processed = process_in_batches(df_norm, batch_size)

# # Save the final DataFrame
# df_norm_processed.to_csv('df_norm_vhl_tail_500.csv', index=False)

# # Display the head of the processed DataFrame
# df_norm_processed.head()


,sample_id,runid,ctrl_cor_frac,pos_ctrl_after_correction,pred_type,CRC,LUNG,BREAST,BLADDER,AGGREGATED,CRC_TF,LUNG_TF,BREAST_TF,BLADDER_TF,AGGREGATED_TF,cohort,request_id,patient_id,visit_name,Alt-NoAlt,SUCCESS,Sample_comment,max_maf_pct_report,RUO_methylation_status,RUO_methylation_tf,TMB_score,TMB_category,MSI_High,input_ng,Plasma_ml_input,Bloodcoll_date,raw_cancer_type,cancer_type,cfDNA_ng_yield,subset,genomic_method,fc_name,cram_path,somatic_max_maf_gene,mut_aa,somatic_max_maf_pct,is_titration,sample_pool,batch,category,gender,age,subtype,treatment,recur,baseline_cancer_type,comment,qc,stage,ethnicity,screening_method,ER_status,HER2_status,ER/HER2_method,comorbidity,smoking_status,s3_panelwide_gene,s3_aa,somatic_maxmafpct_panelwide,somatic_source,mcm_gene,mcm_aa,g360_gene,g360_aa,somatic_driver_pct,somatic_driver_gene,hyper_residual_count_pos_ctrl,panel,msi_status,msi_score,mbd_hyper_file_path,VHL_promoter_methylation_detected,sum_VHL_promoter_mol,methyl_score_VHL
22611,FB09658153-1,NaN,1.0,98126.0,cancer_free,-12.92244,-9.17075,-12.52557,-6.28710,-6.28710,0.0,0.0,0.0,0.0,0.0,CUB_01,NaN,100225709,pre-op,NaN,NaN,NaN,0.56,NaN,NaN,NaN,NaN,NaN,25.657933,NaN,NaN,NaN,CANCER_FREE,NaN,mcm,Shield,NaN,/screening/archive/groups/lunar/flowcells/processed/devel/220415_A01194_0214_AH7MVCDMXY.devel_methylome_v2_8.MethylomeT123v2_set6barcodes.220418_130205/FB09658153-1.pre-realigned.cram,EGFR,A767_V769dup,0.56,no,NaN,LTO_0496,other,NaN,56.0,NaN,NaN,NaN,NaN,NaN,SUCCESS,grade_6,NaN,colonscopy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MCM,NaN,NaN,/screening/archive/groups/lunar/flowcells/processed/devel/220415_A01194_0214_AH7MVCDMXY.devel_methylome_v2_8.MethylomeT123v2_set6barcodes.220418_130205/FB09658153-1.mbd_hyper_molecules.tsv,1,15,0.000155
22613,FB09658165-1,NaN,1.0,148692.0,cancer_free,-13.94278,-8.45393,-10.92956,-7.58447,-8.45393,0.0,0.0,0.0,0.0,0.0,CUB_01,NaN,100224373,pre-op,NaN,NaN,NaN,0.25,NaN,NaN,NaN,NaN,NaN,34.501710,NaN,NaN,NaN,CANCER_FREE,NaN,mcm,Shield,NaN,/screening/archive/groups/lunar/flowcells/processed/devel/220415_A01194_0214_AH7MVCDMXY.devel_methylome_v2_8.MethylomeT123v2_set6barcodes.220418_130205/FB09658165-1.pre-realigned.cram,EGFR,A767_V769dup,0.25,no,NaN,LTO_0496,other,NaN,46.0,NaN,NaN,NaN,NaN,NaN,SUCCESS,grade_6,NaN,colonscopy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MCM,NaN,NaN,/screening/archive/groups/lunar/flowcells/processed/devel/220415_A01194_0214_AH7MVCDMXY.devel_methylome_v2_8.MethylomeT123v2_set6barcodes.220418_130205/FB09658165-1.mbd_hyper_molecules.tsv,0,3,0.000020
22614,FB09658170-1,NaN,1.0,79510.0,cancer_free,-10.62274,-5.30913,-7.66799,-6.45553,-5.30913,0.0,0.0,0.0,0.0,0.0,CUB_01,NaN,100225676,pre-op,NaN,NaN,NaN,0.39,NaN,NaN,NaN,NaN,NaN,35.418677,NaN,NaN,NaN,CANCER_FREE,NaN,mcm,Shield,NaN,/screening/archive/groups/lunar/flowcells/processed/devel/220415_A01194_0214_AH7MVCDMXY.devel_methylome_v2_8.MethylomeT123v2_set6barcodes.220418_130205/FB09658170-1.pre-realigned.cram,EGFR,A767_V769dup,0.39,no,NaN,LTO_0496,other,NaN,52.0,NaN,NaN,NaN,NaN,NaN,SUCCESS,grade_6,NaN,colonscopy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MCM,NaN,NaN,/screening/archive/groups/lunar/flowcells/processed/devel/220415_A01194_0214_AH7MVCDMXY.devel_methylome_v2_8.MethylomeT123v2_set6barcodes.220418_130205/FB09658170-1.mbd_hyper_molecules.tsv,1,13,0.000167
22615,FB09658171-1,NaN,1.0,152277.0,cancer_free,-13.67893,-9.41286,-10.86642,-6.58057,-6.58057,0.0,0.0,0.0,0.0,0.0,CUB_01,NaN,100242445,pre-op,NaN,NaN,NaN,0.15,NaN,NaN,NaN,NaN,NaN,47.359027,NaN,NaN,NaN,CANCER_FREE,NaN,mcm,Shield,NaN,/screening/archive/groups/lunar/flowcells/processed/devel/220415_A01194_0214_AH7MVCDMXY.devel_methylome_v2_8.MethylomeT123v2_set6barcodes.220418_130205/FB09658171-1.pre-realigned.cram,EGFR,A767_V769dup,0.15,no,NaN,LTO_0496,other,NaN,65.0,NaN,NaN,NaN,NaN,NaN,SUCCESS,grade_6,NaN,colonscopy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MCM,NaN,NaN,/screening/archive/gr

In [7]:
# # Function to calculate methylation detected and methyl score
# def calculate_methylation_detected_and_methyl_score(mbd_hyper_file_path, sample_id):
#     # Check if the mbd_hyper file exists
#     if not os.path.exists(mbd_hyper_file_path):
#         print(f"No file found at {mbd_hyper_file_path}")
#         return np.nan, np.nan, np.nan

#     try:
#         # Read the mbd_hyper file into a DataFrame
#         df_mbd_hyper = pd.read_table(mbd_hyper_file_path)

#         # Apply the primary filtering conditions
#         filtered_df = df_mbd_hyper[(df_mbd_hyper['chrom'] == 3) &
#                                    (df_mbd_hyper['start'] >= 10178462) &
#                                    (df_mbd_hyper['start'] <= 10183462) &
#                                    ((df_mbd_hyper['end'] - df_mbd_hyper['start']) < 400) &
#                                    (df_mbd_hyper['mean_mapq'] >= 60) &
#                                    (df_mbd_hyper['cg_count'] >= 10)]
        
#         # Calculate the sum of 'num_molecules'
#         sum_num_molecules = filtered_df['num_molecules'].sum()

#         # Define the path for the secondary file
#         msre_call_rerun_by_sai = "/ghess/omni_v2/users/schen/methylation_data/msre_call/"
#         secondary_file_name = f"{sample_id}.msre_all_call.hdr.tsv"

#         subfolder = os.popen(f"ls -d {msre_call_rerun_by_sai}*/{sample_id}").read().strip()
#         secondary_file_path = os.path.join(subfolder, secondary_file_name)

#         # Check if the secondary file exists
#         if not os.path.exists(secondary_file_path):
#             print(f"No secondary file found at {secondary_file_path}")
#             return 0, sum_num_molecules, np.nan

#         # Read the secondary file
#         df_secondary = pd.read_table(secondary_file_path)
#         hyper_residual_count_pos_ctrl = df_secondary['hyper_residual_count_pos_ctrl'].iloc[0]

#         # Calculate methyl_score
#         methyl_score = sum_num_molecules / hyper_residual_count_pos_ctrl if hyper_residual_count_pos_ctrl else np.nan

#         # Determine the value for 'VHL_promoter_methylation_detected'
#         methylation_detected = 1 if methyl_score > 8.3e-05 and sum_num_molecules > 10 else 0
#         return methylation_detected, sum_num_molecules, methyl_score

#     except Exception as e:
#         print(f"Error processing file {mbd_hyper_file_path}: {e}")
#         return np.nan, np.nan, np.nan

# # Function to process a chunk of the DataFrame
# def process_df_chunk(df_chunk):
#     results = df_chunk.apply(lambda row: calculate_methylation_detected_and_methyl_score(row['mbd_hyper_file_path'], row['sample_id']), axis=1)
#     df_chunk['VHL_promoter_methylation_detected'], df_chunk['sum_VHL_promoter_mol'], df_chunk['methyl_score_VHL'] = zip(*results)
#     return df_chunk

# # Function to process the DataFrame in batches and save each batch
# def process_in_batches_and_save(df, batch_size):
#     for i, start in enumerate(range(0, len(df), batch_size)):
#         end = min(start + batch_size, len(df))
#         df_batch = df[start:end]
#         processed_batch = process_df_chunk(df_batch)
        
#         # Save each batch to a separate CSV file
#         batch_file_name = f'df_norm_vhl_batch_{i}.csv'
#         processed_batch.to_csv(batch_file_name, index=False)
#         print(f"Batch {i} saved to {batch_file_name}")

# # Define batch size
# batch_size = 500 

# # Process df_norm in batches and save each batch
# process_in_batches_and_save(df_norm, batch_size)

KeyboardInterrupt: 

In [5]:
df = pd.read_csv('df_norm_vhl_tail_500.csv', header=0)

# Calculate the percentage of 1s in the 'VHL_promoter_methylation_detected' column
percentage_of_ones = (df['VHL_promoter_methylation_detected'].sum() / len(df)) * 100
percentage_greater_than_10 = (df['sum_VHL_promoter_mol'] > 10).sum() / len(df) * 100

print(f"Percentage of 1s in 'VHL_promoter_methylation_detected': {percentage_of_ones:.2f}%")
print(f"Percentage of mol > 10 in 'sum_VHL_promoter_mol': {percentage_greater_than_10:.2f}%")


Percentage of 1s in 'VHL_promoter_methylation_detected': 19.80%
Percentage of mol > 10 in 'sum_VHL_promoter_mol': 33.00%


In [ ]:
# import epicaller
# import epicaller.methylation.methyl_caller

# def run_mb_crc_caller_wrapper_slurm(sample_meta_df, rundir):
#     region_file='/ghds/groups/labdesk/bshih/clinical_genomics/bims_testing/bi-variant-monitoring/rundeck_monitoring_scripts/CRC_methyl_caller_files/v05_classification_space.tsv'
#     control_region_file='/ghds/groups/labdesk/bshih/clinical_genomics/bims_testing/bi-variant-monitoring/rundeck_monitoring_scripts/CRC_methyl_caller_files/v05_positive_control_regions.tsv'
#     lr_model_file='/ghds/groups/labdesk/bshih/clinical_genomics/bims_testing/bi-variant-monitoring/rundeck_monitoring_scripts/CRC_methyl_caller_files/msre_lr_model.tsv'
#     mr_model_file='/ghds/groups/labdesk/bshih/clinical_genomics/bims_testing/bi-variant-monitoring/rundeck_monitoring_scripts/CRC_methyl_caller_files/msre_mr_model.tsv'
    
#     for _, row in sample_meta_df.iterrows():
#         run_sample_id = row["run_sample_id"]
#         fc_dir = row["fc_dir"]
#         flowcell_id = row["runid"]
#         mbd_hyper_molecules = f"{fc_dir}/{run_sample_id}/{run_sample_id}.mbd_hyper_molecules.tsv"
#         # mbd_residual_molecules = f"{fc_dir}/{run_sample_id}/{run_sample_id}.mbd_residual_molecules.tsv"
        
#         # Check if the molecule file exists
#         if not os.path.exists(mbd_hyper_molecules):
#             continue
        
#         # Check if the job folder exists
#         if not(os.path.exists(f"{rundir}/MB_CRC_jobs")):
#             os.mkdir(f"{rundir}/MB_CRC_jobs")

#         job_file = f"{rundir}/MB_CRC_jobs/MB_call_{run_sample_id}.job" 
#         shebang_content = (f"#!/bin/bash\n"
#                            f"#SBATCH --job-name=MB_call_{run_sample_id}\n"
#                            f"#SBATCH --nodes=1\n"
#                            f"#SBATCH --ntasks=1\n"
#                            f"#SBATCH --cpus-per-task=1\n"
#                            f"#SBATCH --mem=30gb\n"
#                            f"#SBATCH --partition=dev\n"
#                            f"#SBATCH --time=00:10:00\n"
#                            f"#SBATCH --output={rundir}/MB_CRC_jobs/MB_call_{run_sample_id}.log\n\n")
        
#         with open(job_file, "w") as job_output:
#             job_output.write(shebang_content)
#             job_output.write("python3 -m epicaller.methylation.methyl_caller " + 
#                    f"--hyper-molecule-file {mbd_hyper_molecules} " + 
#                    f"--region-file {region_file} " +
#                    f"--control-region-file {control_region_file} " + 
#                    f"--lr-model-file {lr_model_file} " + 
#                    f"--mr-model-file {mr_model_file} " +
#                    "--min-frag-len 120 --max-frag-len 240 --min-cg-count 12 --max-cg-count 30 --cutsite-count-cutoff 1 --min-mapq 60 " + 
#                    f"--sample-id {run_sample_id} --flowcell-id {runid} --outdir {rundir}/") 
#             # print(job_output)    
#         os.system(f"sbatch {job_file}")

In [9]:
# def submit_slurm_job(notebook_path, output_log_dir, job_name="notebook_job", time_limit="00:10:00", mem="30gb"):
#     # Create job script content
#     job_script = f"""#!/bin/bash
# #SBATCH --job-name={job_name}
# #SBATCH --output={output_log_dir}/{job_name}.log
# #SBATCH --nodes=1
# #SBATCH --ntasks=1
# #SBATCH --cpus-per-task=1
# #SBATCH --mem={mem}
# #SBATCH --time={time_limit}
# #SBATCH --partition=dev

# module load jupyter  # Load the necessary module, adjust if needed
# jupyter nbconvert --to notebook --execute {notebook_path} --output {notebook_path}_executed.ipynb
# """

#     # Create a temporary job script file
#     job_script_file = f"{job_name}.job"
#     with open(job_script_file, "w") as file:
#         file.write(job_script)

#     # Submit the job script to Slurm
#     os.system(f"sbatch {job_script_file}")

# # Path to your Jupyter Notebook
# notebook_path = "VHL_promoter_methylation_12122023.ipynb"

# # Directory where you want to save the Slurm log files
# output_log_dir = "/home/hazhang/projects/projects/VHL_promoter_methylation_12122023"

# # Submit the job
# submit_slurm_job(notebook_path, output_log_dir)


,sample_id,runid,ctrl_cor_frac,pos_ctrl_after_correction,pred_type,CRC,LUNG,BREAST,BLADDER,AGGREGATED,CRC_TF,LUNG_TF,BREAST_TF,BLADDER_TF,AGGREGATED_TF,cohort,request_id,patient_id,visit_name,Alt-NoAlt,SUCCESS,Sample_comment,max_maf_pct_report,RUO_methylation_status,RUO_methylation_tf,TMB_score,TMB_category,MSI_High,input_ng,Plasma_ml_input,Bloodcoll_date,raw_cancer_type,cancer_type,cfDNA_ng_yield,subset,genomic_method,fc_name,cram_path,somatic_max_maf_gene,mut_aa,somatic_max_maf_pct,is_titration,sample_pool,batch,category,gender,age,subtype,treatment,recur,baseline_cancer_type,comment,qc,stage,ethnicity,screening_method,ER_status,HER2_status,ER/HER2_method,comorbidity,smoking_status,s3_panelwide_gene,s3_aa,somatic_maxmafpct_panelwide,somatic_source,mcm_gene,mcm_aa,g360_gene,g360_aa,somatic_driver_pct,somatic_driver_gene,hyper_residual_count_pos_ctrl,panel,msi_status,msi_score,mbd_hyper_file_path,VHL_promoter_methylation_detected,sum_VHL_promoter_mol,methyl_score_VHL
0,FB09658153-1,NaN,1.0,98126.0,cancer_free,-12.92244,-9.17075,-12.52557,-6.28710,-6.28710,0.0,0.0,0.0,0.0,0.0,CUB_01,NaN,100225709,pre-op,NaN,NaN,NaN,0.56,NaN,NaN,NaN,NaN,NaN,25.657933,NaN,NaN,NaN,CANCER_FREE,NaN,mcm,Shield,NaN,/screening/archive/groups/lunar/flowcells/processed/devel/220415_A01194_0214_AH7MVCDMXY.devel_methylome_v2_8.MethylomeT123v2_set6barcodes.220418_130205/FB09658153-1.pre-realigned.cram,EGFR,A767_V769dup,0.56,no,NaN,LTO_0496,other,NaN,56.0,NaN,NaN,NaN,NaN,NaN,SUCCESS,grade_6,NaN,colonscopy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MCM,NaN,NaN,/screening/archive/groups/lunar/flowcells/processed/devel/220415_A01194_0214_AH7MVCDMXY.devel_methylome_v2_8.MethylomeT123v2_set6barcodes.220418_130205/FB09658153-1.mbd_hyper_molecules.tsv,1,15,0.000155
1,FB09658165-1,NaN,1.0,148692.0,cancer_free,-13.94278,-8.45393,-10.92956,-7.58447,-8.45393,0.0,0.0,0.0,0.0,0.0,CUB_01,NaN,100224373,pre-op,NaN,NaN,NaN,0.25,NaN,NaN,NaN,NaN,NaN,34.501710,NaN,NaN,NaN,CANCER_FREE,NaN,mcm,Shield,NaN,/screening/archive/groups/lunar/flowcells/processed/devel/220415_A01194_0214_AH7MVCDMXY.devel_methylome_v2_8.MethylomeT123v2_set6barcodes.220418_130205/FB09658165-1.pre-realigned.cram,EGFR,A767_V769dup,0.25,no,NaN,LTO_0496,other,NaN,46.0,NaN,NaN,NaN,NaN,NaN,SUCCESS,grade_6,NaN,colonscopy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MCM,NaN,NaN,/screening/archive/groups/lunar/flowcells/processed/devel/220415_A01194_0214_AH7MVCDMXY.devel_methylome_v2_8.MethylomeT123v2_set6barcodes.220418_130205/FB09658165-1.mbd_hyper_molecules.tsv,0,3,0.000020
2,FB09658170-1,NaN,1.0,79510.0,cancer_free,-10.62274,-5.30913,-7.66799,-6.45553,-5.30913,0.0,0.0,0.0,0.0,0.0,CUB_01,NaN,100225676,pre-op,NaN,NaN,NaN,0.39,NaN,NaN,NaN,NaN,NaN,35.418677,NaN,NaN,NaN,CANCER_FREE,NaN,mcm,Shield,NaN,/screening/archive/groups/lunar/flowcells/processed/devel/220415_A01194_0214_AH7MVCDMXY.devel_methylome_v2_8.MethylomeT123v2_set6barcodes.220418_130205/FB09658170-1.pre-realigned.cram,EGFR,A767_V769dup,0.39,no,NaN,LTO_0496,other,NaN,52.0,NaN,NaN,NaN,NaN,NaN,SUCCESS,grade_6,NaN,colonscopy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MCM,NaN,NaN,/screening/archive/groups/lunar/flowcells/processed/devel/220415_A01194_0214_AH7MVCDMXY.devel_methylome_v2_8.MethylomeT123v2_set6barcodes.220418_130205/FB09658170-1.mbd_hyper_molecules.tsv,1,13,0.000167
3,FB09658171-1,NaN,1.0,152277.0,cancer_free,-13.67893,-9.41286,-10.86642,-6.58057,-6.58057,0.0,0.0,0.0,0.0,0.0,CUB_01,NaN,100242445,pre-op,NaN,NaN,NaN,0.15,NaN,NaN,NaN,NaN,NaN,47.359027,NaN,NaN,NaN,CANCER_FREE,NaN,mcm,Shield,NaN,/screening/archive/groups/lunar/flowcells/processed/devel/220415_A01194_0214_AH7MVCDMXY.devel_methylome_v2_8.MethylomeT123v2_set6barcodes.220418_130205/FB09658171-1.pre-realigned.cram,EGFR,A767_V769dup,0.15,no,NaN,LTO_0496,other,NaN,65.0,NaN,NaN,NaN,NaN,NaN,SUCCESS,grade_6,NaN,colonscopy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MCM,NaN,NaN,/screening/archive/groups/lunar/flowc

In [ ]:
# #used this as WTF to find all the S3 fc_dir as the original WTF script missed some folder
# import pandas as pd
# import glob

# # Read the CSV file
# ghdb = pd.read_csv(f"{work_dir}/ghdb.csv", header=0)

# # Define the path to the 'flowcentral' directory
# fc_all_folder = "/ghds/ivd/flowcentral"

# # Add a new column 'fc_dir' to the DataFrame with default empty values
# ghdb['fc_dir'] = ''

# # Iterate over each row of the DataFrame
# for index, row in ghdb.iterrows():
#     try:
#         runid = row["runid"]
#         pattern = f"{fc_all_folder}/*{runid}.*"
        
#         # Use the glob function to find folders that match the pattern
#         folders = glob.glob(pattern)
        
#         # Check if any matching folders were found
#         if not folders:
#             print(f"No suitable folder found for {runid}")
#             ghdb.loc[index, "fc_dir"] = ''
#         else:
#             folder = folders[0]
#             ghdb.loc[index, "fc_dir"] = folder

#     except Exception as e:
#         print(f"An error occurred while processing {runid}: {str(e)}")